# Projet Leyenda - Livrable 3 
## I. Introduction
### 1.1 Introduction au projet

L’entreprise TouNum fait appel à nos services afin d’ajouter à leurs prestations un algorithme de Machine Learning permettant d’améliorer les performances de numérisation de leurs clients. Le premier contrat concerne une solution visant à analyser des photographies afin d’en écrire une légende descriptive automatique, en suivant le concept de captioning.

Cette mission implique plusieurs enjeux : dans un premier temps, les images étant de qualité, définition et résolution pouvant être variables, il faudra nettoyer l’image. De plus, comme ceux-ci comptent travailler sur des types de documents différents, il sera important de savoir trier les documents en question.

![](https://i.ibb.co/jRY8m0y/workflow-projet.png)

Ci-dessus, le Workflow fourni par Tounum. Pour implémenter les algorithmes, nous utiliserons Python (sous forme d’un Notebook Jupyter) ainsi que les librairies SciKit et TensorFlow. D’un point de vue plus mathématique, nous utiliserons également NumPy et MatPlotLib.

Tounum nous fournit également un Dataset dans lequel des images déjà catégorisées permettront d’entraîner nos algorithmes. Dans un premier temps, nous effectuerons une classification binaire via un réseau de neurones afin de classer les fichiers en 2 classes : photos et autres images. Il faudra donc différencier les photos des schémas ainsi que des peintures par exemple.

Pour le captioning, 2 techniques de Machine Learning sont à utiliser. Dans un premier temps, les réseaux de neurones convolutifs, CNN, afin de prétraiter les images. Par la suite, des réseaux de neurones récurrents, RNN, pour générer les étiquettes (descriptions).

Une autre contrainte est à prendre en compte, notamment l’automatisation complète du processus, de l’insertion du jeu de données jusqu’à la création des étiquettes.

3 livrables seront à rendre respectivement fin de semaine 3, fin de semaine 4 et fin de semaine 5 :

- Livrable 1 : Classification Binaire.

- Livrable 2 : Traitement d’images.

- **Livrable 3 : Captioning d’images**.

Dans ce notebook, vous trouverez le code ainsi que les justifications techniques complètes concernant le livrable 3 sur le captionning d'images. 

Afin de simplifier la compréhension de celui-ci, nous reprendrons progressivement les résultats des 2 premiers livrables afin de faire de ce livrable 3 un Notebook permettant de reprendre l'entièreté du projet.

### 1.2 Notre équipe de Bordeaux

L'équipe est composée d'étudiants de CESI en option **Data Science** composée de :
- Elise Beaumatin
- Axel Brosset
- Gaylord Giret
- Mathieu Musard
- Xavier Labarbe

### 1.3 Introduction au livrable 3 : Captioning

Ce notebook nous permettra de présenter le travail effectué sur le livrable 3. L'objectif de celui-ci est la création d'un réseau de neurones générant des légendes pour des photographies. Le jeu de données fourni par Tounum est le dataset MS COCO. Deux parties distinctes sont à prévoir : 
- La partie CNN, encodant les images en une représentation interne.
- La partie RNN, utilisant cette même représentation pour prédire l'annotation séquence par séquence.

A noter également que les images seront prétraitées par un CNN pré-entrainé pour la classification des images.

## II. Remise en contexte

### 2.1 Livrable 1

#### EfficientNetV2B0

Dans le premier livrable, nous avons décidé d'utiliser le modèle de Transfert Learning EfficientNetV2B0. Il s'agit d'un modèle de reconnaissance d'images pré-entraînées développé par Google et publié en 2021, qui utilise une architecture de réseau de neurones convolutionnel (CNN) optimisée pour être efficace en termes de consommation de ressources informatiques tout en conservant une haute performance. Il a été entraîné sur la base de données ImageNet et a été utilisé avec succès dans de nombreuses tâches de vision par ordinateur, telles que la détection d'objets, la classification d'images et la génération de descriptions de contenu d'image. Ce réseau de neurones a été entraîné sur la base de données ImageNet.

Le transfert learning est une technique qui permet de réutiliser les connaissances acquises par un modèle pré-entraîné sur une tâche donnée pour entraîner un nouveau modèle sur une tâche différente. Cela peut être utile lorsqu'il y a peu de données disponibles pour entraîner un modèle sur la tâche spécifique, ou lorsqu'il est difficile de collecter ces données. Le transfert learning peut également aider à améliorer les performances d'un modèle sur une tâche spécifique en utilisant les connaissances acquises par un modèle pré-entraîné sur une tâche similaire.

#### Détail des couches de notre modèle

La couche 1 de notre réseau (Rescaling) est une étape de prétraitement permettant de normaliser toutes nos images. Ainsi, les valeurs de celles-ci seront comprises entre 0 et 1, évitant au modèle de privilégier certaines images à d'autres. Celle-ci n'est donc pas réellement une couche.

La couche 2 de notre réseau (Conv2D) est une étape permettant de d'introduire le principe de convolution. Cette couche est composée de 16 filtres de convolution, sa fonction d'activation est ReLu, et on utilise un padding pour "remplir" les bords de l'entrée avec des valeurs zéro afin de préserver la dimension de l'entrée.

La couche 3 de notre réseau (MaxPooling2D) est une étape permettant de réduire la dimension de l'entrée en sélectionnant le maximum de chaque sous-région de l'entrée.

La couche 4 de notre réseau (Dropout) est une étape de régularisation permettant de "désactiver" aléatoirement un pourcentage de neurones d'une couche pendant l'entraînement, ce qui force le modèle à se baser sur les autres neurones et à éviter de dépendre trop fortement d'un sous-ensemble de neurones. Cette étape n'est donc pas réellement une couche.

Les couches 5 à 10 sont également des trios de couches Conv2D / MaxPooling2D / Dropout. A noter que l'on multiplie par 2 le nombre de filtres de convolution à chaque couche Conv2D afin de spécifier les caractéristiques que ces filtres permettent de détecter ; ainsi, notre dernière couche Conv2D est composé de 64 filtres de convolution.

La couche 11 de notre réseau (Flatten) est une étape permettant de transformer l'information contenue dans une image en vecteur. Ici, chaque composante du vecteur se transforme alors en neurone.

La couche 12 de notre réseau (Dense) est une étape permettant de créer une couche de neurones "pleine", c'est-à-dire que chaque neurone de cette couche est relié à l'ensemble des neurones de la couche précédente. Cette couche est composée de 128 neurones. On utilise également la fonction d'activation ReLu.

La couche 13 de notre réseau (Dropout) est une étape de régularisation permettant de "désactiver" aléatoirement un pourcentage de neurones d'une couche pendant l'entraînement, ce qui force le modèle à se baser sur les autres neurones et à éviter de dépendre trop fortement d'un sous-ensemble de neurones. Cette étape n'est donc pas réellement une couche.

Enfin, la couche 14 de notre réseau (Dense) permet de configurer un neurone unique en sortie. Ce neurone est connecté à l'ensemble des neurones de la couche précédente. Nous utilisons ici la fonction d'activation Sigmoid afin de déterminer une sortie binaire pour notre résultat.

In [ ]:
classification_model = tf.keras.models.load_model('../input/classification/classification.h5');
classification_model.summary();

In [1]:
def load_image(infilename):
    img = tf.io.decode_image(tf.io.read_file(image_path))
    plt.imshow(img)
    plt.show() 
    img = tf.image.resize(img, [224, 244])
    return img

def is_a_photo(img_path):
    img = load_image(img_path)

    pred = classification_model.predict(np.asarray(img))
    print(pred)
    if pred>=0.5:
        print("C'est une photo ! ")
        return True
    else:
        print("Ce n'est pas une photo")
        return False

In [ ]:
is_it_a_photo('../input/image8/Photo__8.jpg')
is_it_a_photo('../input/test-data/Dataset soutenance/Paint.jpg')

### 2.2 Livrable 2 

Le livrable 1 nous a permis d'identifier les photographies parmi un ensemble d'images avec une précision de 99%. 

L'étape suivante était donc de traiter des photographies afin d'améliorer l'efficacité des futurs algorithmes de Captioning. 

Ces différents traitements permettent dans un premier temps de supprimer/d'atténuer les bruits et bruitages : ce nom représente tout parasite ou dégradation que subit une image lors de son acquisition. Par exemple, les défauts d'une photo prise de nuit sans éclairage sont des bruitages. 

Les auto-encodeurs peuvent être utilisés pour réduire le bruit d'une image : en effet, l'entraînement lui permet d'apprendre à identifier les problèmes courants de bruitages et les supprimer/atténuer fortement. 

## 3. Captioning d'images 

Qu'est-ce que le captioning d'images ? Le captioning d'images est le processus de génération automatique de légendes pour des images. Les modèles de traitement du langage naturel, tels que les réseaux de neurones, sont souvent utilisés pour générer les légendes en utilisant des données d'apprentissage d'images et de légendes. Les légendes générées peuvent décrire les objets, les personnes, les actions ou les scènes présentes dans l'image. Le captioning d'images est utilisé dans de nombreuses applications, notamment la reconnaissance d'images, la recherche d'images et l'accessibilité pour les personnes malvoyantes.

Avant d'arriver à cette étape, nous avons déjà : 
- Fait un tri préliminaire des images fournies dans le jeu de données, afin de ne garder que les photographies. 
- Débruité les images afin de les standardiser et de les rendre plus facilement manipulables par l'algorithme de captioning. 

Tounum nous a fourni un jeu de données de 150 images à légender. Le but est que notre algorithme soit capable d'écrire une phrase simple en français permettant de décrire ce qu'il y a sur la photo.